# CCI STAC - Demonstration

The ``pystac_client`` package can be used to search and access the data with in the CCI STAC. This notebook is a showcase of some of the methods which can be used.

Note that the pystac_client itself has some [good documentation](https://pystac-client.readthedocs.io/en/latest/usage.html) if any of this isn't clear or something isn't covered. 


## 1. Pystac-client
First we import `pystac_client` library.

In [1]:
import pystac_client
from pystac_client.stac_api_io import StacApiIO

import itertools

Now we can create our client using the url of the STAC catalog that we want to work with.

In [3]:
client = pystac_client.Client.open("https://api.stac.164.30.69.113.nip.io/")

The client has many different function to examine the collections available we can use the `get_collections()` method, because there are a lot of collections available we only display the first 10:

In [4]:
for coll in itertools.islice(client.get_collections(), 10):
    print(f"{coll.id}: {coll.title}")

004fd44ff5124174ad3c03dd2c67d548: ESA Cloud Climate Change Initiative (Cloud_cci): AVHRR-PM monthly gridded cloud properties, version 3.0
004fd44ff5124174ad3c03dd2c67d548-main: (NonDRS) AVHRR-PM monthly gridded cloud properties, version 3.0
00a0da5d759d4d69ae866a23b84011be: ESA Ozone Climate Change Initiative (Ozone CCI): Merged Level 3 Limb Ozone Monthly Zonal Mean (MZM) Profiles, Version 2
00a0da5d759d4d69ae866a23b84011be-main: (NonDRS) ESA Ozone Climate Change Initiative (Ozone CCI): Merged Level 3 Limb Ozone Monthly Zonal Mean (MZM) Profiles, Version 2
00fe090efc58446e8980992a617f632f: ESA Antarctic Ice Sheet Climate Change Initiative (Antarctic_Ice_Sheet_cci): Antarctic Ice Sheet monthly velocity from 2017 to 2020, derived from Sentinel-1, v1
00fe090efc58446e8980992a617f632f-main: (NonDRS) ESA Antarctic Ice Sheet Climate Change Initiative (Antarctic_Ice_Sheet_cci): Antarctic Ice Sheet monthly velocity from 2017 to 2020, derived from Sentinel-1, v1
01b00854797d44a59d57c8cce08821eb:

Alternatively, the STAC browser can be used to nagivate the nested collections: https://radiantearth.github.io/stac-browser/#/external/api.stac.164.30.69.113.nip.io/collections/cci.

Once we have found a collection we want to work with we can use it's ID and the `get_collection()` method to retrieve it.

In [5]:
collection_id = 'esacci.lakes.day.l3s.lk_products.multi-sensor.multi-platform.merged.v2-1-0.r1'
collection = client.get_collection(collection_id)

We can inspect some of the collections metadata including it's spatial and temporal extents, as well as any summary information it has on it's items.

In [6]:
print(collection.extent.spatial.bboxes)
print(collection.extent.temporal.intervals)
print(collection.summaries.to_dict())

[[-180, -90, 180, 90]]
[[datetime.datetime(1992, 9, 26, 0, 0, tzinfo=tzutc()), datetime.datetime(1992, 10, 15, 23, 59, 59, tzinfo=tzutc())]]
{'Assets': [{'name': 'NetCDF', 'common_name': 'NetCDF', 'description': 'None'}]}


We can also use `get_items()` to retrieve all of the items associated to the collection. This can return large numbers of items so it's recommend to use the search API instead - more on that later. 

In [8]:
for item in itertools.islice(collection.get_items(), 10):
    print(item.id)

ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221231-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221230-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221229-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221228-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221227-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221226-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221225-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221224-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221223-fv2.1.0
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221222-fv2.1.0


## 2. Searching

Search allows to to filter items across all collections through their associated metadata. `get_queryables` can be used to see the avaiable fields to search on. These typically include the related collection, spatial and temporal extents, and other item properties.

**Note**: this is currently not returning all queryables so can be skipped for now.

In [17]:
client.get_queryables()

{'$schema': 'https://json-schema.org/draft/2019-09/schema',
 '$id': 'https://stac-api.example.com/queryables',
 'type': 'object',
 'title': 'Queryables for STAC API',
 'description': 'Queryable names for the STAC API Item Search filter.',
 'properties': {'id': {'description': 'ID',
   '$ref': 'https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json#/definitions/core/allOf/2/properties/id'},
  'collection': {'description': 'Collection',
   '$ref': 'https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json#/definitions/core/allOf/2/then/properties/collection'},
  'geometry': {'description': 'Geometry',
   '$ref': 'https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json#/definitions/core/allOf/1/oneOf/0/properties/geometry'},
  'datetime': {'description': 'Acquisition Timestamp',
   '$ref': 'https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/datetime.json#/properties/datetime'},
  'created': {'description': 'Creation Timestamp',
   '$ref': 'https:/

These fields can then be used during search which is outlined [here](https://github.com/radiantearth/stac-api-spec/tree/release/v1.0.0/item-search).

Fields in the core specification for example an item's collection, spatial, and temporal properties can be search through their own parameters in the `client.search()` method, there are two kinds of datetime selection methods we can use:

In [11]:
item_search = client.search(
    collections=[collection_id],
    datetime="2022-12-21/2022-12-31",
    max_items=10,
)

for item in itertools.islice(item_search.items(), 10):
    print(item.id, item.properties["start_datetime"], item.properties["end_datetime"])

ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221231-fv2.1.0 2022-12-31T00:00:00Z 2022-12-31T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221230-fv2.1.0 2022-12-30T00:00:00Z 2022-12-30T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221229-fv2.1.0 2022-12-29T00:00:00Z 2022-12-29T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221228-fv2.1.0 2022-12-28T00:00:00Z 2022-12-28T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221227-fv2.1.0 2022-12-27T00:00:00Z 2022-12-27T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221226-fv2.1.0 2022-12-26T00:00:00Z 2022-12-26T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221225-fv2.1.0 2022-12-25T00:00:00Z 2022-12-25T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221224-fv2.1.0 2022-12-24T00:00:00Z 2022-12-24T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221223-fv2.1.0 2022-12-23T00:00:00Z 2022-12-23T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221222-fv2.1.0 2022-12-22T00:00:00Z 2022-12-22T23:59:59Z


If the catalogue has the filter extenions the `query` parameter in `client.search()` can be used to create more complex search using all available fields and use the CQL query format.

In [12]:
item_search = client.search(
    collections=[collection_id],
    query=[
        'start_datetime>=2022-12-21',
        'end_datetime<=2022-12-31', 
    ],
    max_items=10,
)

for item in itertools.islice(item_search.items(), 10):
    print(item.id, item.properties["start_datetime"], item.properties["end_datetime"])

ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221231-fv2.1.0 2022-12-31T00:00:00Z 2022-12-31T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221230-fv2.1.0 2022-12-30T00:00:00Z 2022-12-30T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221229-fv2.1.0 2022-12-29T00:00:00Z 2022-12-29T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221228-fv2.1.0 2022-12-28T00:00:00Z 2022-12-28T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221227-fv2.1.0 2022-12-27T00:00:00Z 2022-12-27T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221226-fv2.1.0 2022-12-26T00:00:00Z 2022-12-26T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221225-fv2.1.0 2022-12-25T00:00:00Z 2022-12-25T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221224-fv2.1.0 2022-12-24T00:00:00Z 2022-12-24T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221223-fv2.1.0 2022-12-23T00:00:00Z 2022-12-23T23:59:59Z
ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221222-fv2.1.0 2022-12-22T00:00:00Z 2022-12-22T23:59:59Z


Here we've used the query kwarg to specify the start and end datetime for our search range, and these are all _and_-ed together so we get the range we want. Note as well that we've set `max_items` to 10 so we don't end up grabbing every item in the search result, you could remove this if you so desired. We've only specified one collection here but you can specify multiple, just bear in mind you always need to specify your collection(s) as a list - even if there's only one. 

We can interrogate our search results and see how many we matched:

**Note**: Currently the API uses the wrong field name this will be patch in an up coming update.

In [18]:
print(item_search.matched())


None


/home/users/rhys_r_e/.local/lib/python3.11/site-packages/pystac_client/item_search.py:680: UserWarning: numberMatched or context.matched not in response
  warnings.warn("numberMatched or context.matched not in response")


AttributeError: 'ItemSearch' object has no attribute 'to_dict'

Note that we could get to the same result by manually polling the API endpoint with a get request. This could be browsed to on any web browser with the appropriate url, and in python this can be done nice and easily with the requests library. `item_search` also, very conveniently, has a method for pulling the url and query parameters out:

In [13]:
item_search.url_with_parameters()

'https://api.stac.164.30.69.113.nip.io/search?collections=esacci.lakes.day.l3s.lk_products.multi-sensor.multi-platform.merged.v2-1-0.r1&query=%7B%22start_datetime%22%3A%7B%22gte%22%3A%222022-12-21%22%7D%2C%22end_datetime%22%3A%7B%22lte%22%3A%222022-12-31%22%7D%7D'

(NOTE: it doesn't seem to automatically add the limit=10 to limit the max number of items we retrieve, but that's a simple enough addition)
Now we can navigate to that as a url directly or, as previously mentioned, use the `requests` library. 

In [22]:
import requests
response = requests.get(item_search.url_with_parameters() + "&limit=1")

api_item = response.json()["features"][0]

print(api_item["properties"])

{'datetime': None, 'start_datetime': '2022-12-31T00:00:00Z', 'end_datetime': '2022-12-31T23:59:59Z', 'license': 'other', 'version': ['v2.1.0'], 'aggregation': False, 'platforms': ['Sentinel-6A', 'SARAL', 'Jason-2', 'Sentinel-3A', 'Jason-1', 'Jason-3', 'ERS-1', 'GFO', 'ERS-2', 'Envisat', 'Sentinel-3B'], 'collections': ['lakes', '7fc9df8070d34cacab8092e45ef276f1', 'esacci.LAKES.day.L3S.LK_PRODUCTS.multi-sensor.multi-platform.MERGED.v2-1-0.r1'], 'opensearch_url': 'https://archive.opensearch.ceda.ac.uk/opensearch/description.xml?parentIdentifier=7fc9df8070d34cacab8092e45ef276f1', 'esa_url': 'https://climate.esa.int/en/catalogue/7fc9df8070d34cacab8092e45ef276f1/', 'dataType': 'LK_PRODUCTS', 'project': 'LAKES', 'frequency': 'day', 'productString': 'MERGED', 'platformGroup': [], 'productVersion': 'v2.1.0', 'processingLevel': 'L3S', 'drsId': 'esacci.LAKES.day.L3S.LK_PRODUCTS.multi-sensor.multi-platform.MERGED.v2-1-0.r1', 'ecv': 'LAKES', 'datasetId': '7fc9df8070d34cacab8092e45ef276f1', 'institu

This is obviously a little unwieldy in this form and less convenient, but it can be useful going straight to the source. 

If we inspect the properties of the first item we retrieved, we can see the associated metadata.  Every field here can be searched across with the query kwarg in `client.search()` (which we previously did with `start_datetime` and `end_datetime`)

In [16]:
next(item_search.items()).properties

{'datetime': None,
 'start_datetime': '2022-12-31T00:00:00Z',
 'end_datetime': '2022-12-31T23:59:59Z',
 'license': 'other',
 'version': ['v2.1.0'],
 'aggregation': False,
 'platforms': ['Sentinel-6A',
  'SARAL',
  'Jason-2',
  'Sentinel-3A',
  'Jason-1',
  'Jason-3',
  'ERS-1',
  'GFO',
  'ERS-2',
  'Envisat',
  'Sentinel-3B'],
 'collections': ['lakes',
  '7fc9df8070d34cacab8092e45ef276f1',
  'esacci.LAKES.day.L3S.LK_PRODUCTS.multi-sensor.multi-platform.MERGED.v2-1-0.r1'],
 'opensearch_url': 'https://archive.opensearch.ceda.ac.uk/opensearch/description.xml?parentIdentifier=7fc9df8070d34cacab8092e45ef276f1',
 'esa_url': 'https://climate.esa.int/en/catalogue/7fc9df8070d34cacab8092e45ef276f1/',
 'dataType': 'LK_PRODUCTS',
 'project': 'LAKES',
 'frequency': 'day',
 'productString': 'MERGED',
 'platformGroup': [],
 'productVersion': 'v2.1.0',
 'processingLevel': 'L3S',
 'drsId': 'esacci.LAKES.day.L3S.LK_PRODUCTS.multi-sensor.multi-platform.MERGED.v2-1-0.r1',
 'ecv': 'LAKES',
 'datasetId': '

So we can now do a slightly more interesting/complicated search using some of this metadata. First we define a point in space, and the shapely library provides some useful utilities for doing so. Note that the [stac spec](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md#bbox) states that the bbox or coordinates must be specified in order of longitude-lattitude(-elevation) in line with the [GeoJSON standard RFC7946](https://datatracker.ietf.org/doc/html/rfc7946#section-5).

In [23]:
from shapely import Point
point = Point(-1.3144835766058023, 51.57555380377267) # Atlas building at RAL

This is using lattitude and longitude, but there is no required coordinate system built into STAC, so this is dependent entirely on the data you are trying to access. We're also only using a point in this example, but this could be an arbitrary bounding box or shape if you like. 

Lets do a search using this and a little more metadata. We input our spatial data using the `intersects` kwarg, and filter on the `processingLevel` using the `query` kwarg as before.

In [32]:
item_search = client.search(
    collections=[collection_id],
    intersects=point,
    query=[
        'start_datetime>=2022-12-21',
        'end_datetime<=2022-12-31',
        'processingLevel=L3S'
    ],
    max_items=10,
)

In [33]:
# item_search.matched()
items = item_search.items()

<generator object ItemSearch.items at 0x7fac95d847c0>

So we've narrowed down quite a bit further. If you want more information on alternative ways to search or query there is good documentation available online, e.g. [here](https://pystac-client.readthedocs.io/en/latest/usage.html#query-extension), [here](https://github.com/stac-api-extensions/query), and [here](https://github.com/radiantearth/stac-api-spec/blob/release/v1.0.0/item-search/examples.md).

## 3. From items to assets

Now we have some items that we're keen on, we can take a closer look and actually load some of the data. First lets have a look at the json object. 

In [34]:
item = next(items)
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221231-fv2.1.0',
 'properties': {'datetime': None,
  'start_datetime': '2022-12-31T00:00:00Z',
  'end_datetime': '2022-12-31T23:59:59Z',
  'license': 'other',
  'version': ['v2.1.0'],
  'aggregation': False,
  'platforms': ['Sentinel-6A',
   'SARAL',
   'Jason-2',
   'Sentinel-3A',
   'Jason-1',
   'Jason-3',
   'ERS-1',
   'GFO',
   'ERS-2',
   'Envisat',
   'Sentinel-3B'],
  'collections': ['lakes',
   '7fc9df8070d34cacab8092e45ef276f1',
   'esacci.LAKES.day.L3S.LK_PRODUCTS.multi-sensor.multi-platform.MERGED.v2-1-0.r1'],
  'opensearch_url': 'https://archive.opensearch.ceda.ac.uk/opensearch/description.xml?parentIdentifier=7fc9df8070d34cacab8092e45ef276f1',
  'esa_url': 'https://climate.esa.int/en/catalogue/7fc9df8070d34cacab8092e45ef276f1/',
  'dataType': 'LK_PRODUCTS',
  'project': 'LAKES',
  'frequency': 'day',
  'productString': 'MERGED',
  'platformGroup': [],
  'productVersion': 'v2.1.0',

There's a lot there, feel free to browse at your own leisure. 

If we're interested in getting to the data we need to specifically look at the assets dictionary.

In [35]:
for key, value in item.assets.items():
    print(key, value.href)

NetCDF https://dap.ceda.ac.uk/neodc/esacci/lakes/data/lake_products/L3S/v2.1/merged_product/2022/12/ESACCI-LAKES-L3S-LK_PRODUCTS-MERGED-20221231-fv2.1.0.nc


The NetCDF file can be opened using xarray.

In [37]:
import xarray as xr

ds = xr.open_dataset(f"{item.assets['NetCDF'].href}#mode=bytes")

---
## Kerchunk example

The following is an example kerchunk file that has been generated for some specific CCI data, which can be used to index only the parts of the netcdf file you want to work with as opposed to the whole thing. To do this we use fsspec to read the kerchunk file as a virtual zarr file, and then open it using xarray's `open_zarr()` method. We can then select a subset of data and it works in a fraction of the time doing something equivalent with the above `open_dataset` method would take.


In [40]:
import xarray as xr

kerchunk_item_search = client.search(
    query=[
        'aggregation=true'
    ],
    max_items=10,
)

kerchunk_items = kerchunk_item_search.items()
kerchunk_item = next(kerchunk_items)

print(kerchunk_item)
print(kerchunk_item.assets['reference_file'].href)
kerchunk_ds = xr.open_dataset(kerchunk_item.assets['reference_file'].href, engine='kerchunk')

<Item id=pctCCI_mon_ESACCI-CLOUD-ATSR2-AATSR-3-0_BE_gr_199501-201212_kr1.0>
https://dap.ceda.ac.uk/neodc/esacci/cloud/metadata/kerchunk/obs4MIPs/DWD/ESACCI-CLOUD-ATSR2-AATSR-3-0/mon/pctCCI/gr/v20200106/pctCCI_mon_ESACCI-CLOUD-ATSR2-AATSR-3-0_BE_gr_199501-201212_kr1.0.json


Then we can inspect our dataset and plot something from it

In [41]:
list(kerchunk_ds.variables)

['lat', 'lat_bnds', 'lon', 'lon_bnds', 'pctCCI', 'time', 'time_bnds']